# Pain Charts classification with CNN

Idea: Creation of a simple a classifier for the groups found in Boudreau et al 2018

- Data from Patellofemoral Pain Patients
- Groups defined by a unsupervised cluster classfication (K-means)

Method: Application of a CNN with a simple architecture and build up from here
     
Reference: Boudreau, S.A., Royo, A.C., Matthews, M. et al. Distinct patterns of variation in the distribution of knee pain. Sci Rep 8, 16522 (2018). www.nature.com/articles/s41598-018-34950-2

In [20]:
#Import of the the main 
import sklearn
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import numpy as np
import glob
import cv2


import tensorflow as tf

from keras import layers
from keras.layers import Input, Add, Dense, Activation,ZeroPadding2D, ZeroPadding1D, BatchNormalization, Flatten, Conv1D, Conv2D, AveragePooling2D, AveragePooling1D,MaxPooling2D, MaxPooling1D, GlobalMaxPooling1D,GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils, to_categorical   
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import lmdb
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [3]:
# Let's import the file names from our dataset
fileNames = glob.glob(r'C:\Users\Albert\Documents\GitHub\PC data\*.png')
print('Number of files imported is',len(fileNames))
kernel = np.array([[0,1,0],[1,1,1],[0,1,0]],np.uint8)
binIms = []   
PainChartList = []

for fileName in fileNames:
    im = cv2.imread(fileName)
    binIm = np.zeros(np.shape(im)[0:2])
    #Creating a mask from the pain Image, this will be the binarized version of the image
    binIm[(im[:,:,2]>150) & (im[:,:,0]==0)] = 1
    #Open the pain region so small empty spaces within the pain region are filled
    binIm = cv2.morphologyEx(binIm, cv2.MORPH_OPEN, kernel)
    #Saving images in a List 
    PainChartList.append(binIm)

Number of files imported is 299


In [4]:
y = np.loadtxt('C:/Users/Albert/Documents/GitHub/classesK4')

In [5]:
arrayPain = np.asarray(PainChartList)
arrayPainCropped = arrayPain[:,int(arrayPain.shape[1]/2):int(arrayPain.shape[1]),:]
X_train, X_test, Y_train, Y_test = train_test_split(arrayPainCropped, y, test_size=0.3)
X_trainR = np.expand_dims(X_train,3)
X_testR = np.expand_dims(X_test,3)

In [13]:
def PainNet(input_shape = (568, 447, 1), classes = 4):
    """
    Implementation of PainNet the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL 
    
    Arguments:
    input_shape -- shape of the images of the dataset (?,568, 447, 1)
    classes -- integer, number of classes (?,4)

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
   
    # Stage 1
    X = Conv2D(32, (30,30), strides = (3, 3), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X) #(228, 168, 64)
    print('Conv1:',X.shape)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X) #114, 84, 64
    #print('Pool1:',X.shape)
    
    # Stage 2
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv2', kernel_initializer = glorot_uniform(seed=0))(X) #(50,35,128)
    print('Conv2:',X.shape)
    X = BatchNormalization(axis = 3, name = 'bn_conv2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    #print('Pool2:',X.shape)
    
     # Stage 2
    X = Conv2D(128, (2, 2), strides = (2, 2), name = 'conv3', kernel_initializer = glorot_uniform(seed=0))(X) #(50,35,128)
    print('Conv2:',X.shape)
    X = BatchNormalization(axis = 3, name = 'bn_conv3')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)
    #print('Pool2:',X.shape)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='PainNet')

    return model

In [14]:
model = PainNet(input_shape = X_trainR.shape[1:], classes = 4)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Conv1: (None, 182, 142, 32)
Conv2: (None, 42, 32, 64)
Conv2: (None, 10, 7, 128)


In [15]:
Y_train = to_categorical(Y_train.T, num_classes=4)
Y_test = to_categorical(Y_test.T, num_classes=4)
print ("number of training examples = " + str(X_trainR.shape[0]))
print ("number of test examples = " + str(X_testR.shape[0]))
print ("X_trainR shape: " + str(X_trainR.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_testR shape: " + str(X_testR.shape))
print ("Y_test shape: " + str(Y_test.shape))
conv_layers = {}

number of training examples = 209
number of test examples = 90
X_trainR shape: (209, 568, 447, 1)
Y_train shape: (209, 4)
X_testR shape: (90, 568, 447, 1)
Y_test shape: (90, 4)


In [16]:
model.fit(X_trainR,Y_train,epochs = 5, batch_size = 16, shuffle = True)

Epoch 1/5
209/209 [==============================] - 60s 288ms/step - loss: 1.4918 - accuracy: 0.5311
Epoch 2/5
209/209 [==============================] - 46s 219ms/step - loss: 1.2554 - accuracy: 0.5742
Epoch 3/5
209/209 [==============================] - 46s 220ms/step - loss: 1.1262 - accuracy: 0.5837
Epoch 4/5
209/209 [==============================] - 53s 255ms/step - loss: 1.0550 - accuracy: 0.6077
Epoch 5/5
209/209 [==============================] - 50s 237ms/step - loss: 1.1360 - accuracy: 0.5933


In [17]:
from keras.applications import ResNet50

In [25]:
base_model = ResNet50(weights = 'imagenet',include_top = False,input_shape = (568, 447, 1))

ValueError: The input must have 3 channels; got `input_shape=(568, 447, 1)`

In [21]:
x=base_model.output
x=AveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(4,activation='softmax')(x) #final layer with softmax activation

In [22]:
model=Model(inputs=base_model.input,outputs=preds)

In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_trainR,Y_train,epochs = 5, batch_size = 16, shuffle = True)

ValueError: Error when checking input: expected input_6 to have shape (None, None, 3) but got array with shape (568, 447, 1)